# Progetto Computer Science

*Nome:* Christian

*Cognome:* Uccheddu

*Matricola:* 800428


**Setup dell'ambiente**

Importazione di tutte le librerie utili al funzionamento del programma e settaggio dei path in cui sono presenti i dati in modo da essere facilmente cambiabili all'inizio del programma.

In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
loans_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans.csv"
loans_lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans_lenders.csv"
lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/lenders.csv"
country_stats_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/country_stats.csv" 

**1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.**

Per questo compito mi occupo di importare il file loans_lenders, di analizzarne la struttura e successivamente decidere come procedere.

In [3]:
loans_lenders = pd.read_csv(loans_lenders_path)

In [4]:
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


Vedendo la struttura del dataframe, la normalizzazione della tabella consiste nel separare gli elementi presenti nella colonna lenders in modo che il dataframe sia formato da celle in cui è presente solo una istanza. Per farlo separo gli elementi che si trovano in corrispondenza della virgola, essendoci uno spazio vuoto dopo la virgola dovrò stare attento a considerarlo. Per ovviare a questo problema trasformo gli elementi ad ogni istanza di lenders in una lista.

In [6]:
loans_lenders['lenders'] = loans_lenders.apply(lambda row: row['lenders'].split(','), axis = 1)

Per separare gli elementi utilizzo la funzione *explode* implementata all'interno della libreria pandas che permette di sepoarare gli elementi di una lista lungo l'asse voluto.

In [7]:
loans_lenders_norm = loans_lenders.explode('lenders')

In [8]:
# Decommentare questa riga per vedere i risultati.
#loans_lenders_norm.head()

**2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.**

Come per il punto precedente procedo con una esplorazione preliminare del file in cui sono presenti gli attributi utili allo svolgimento del compito. Dopo averne fatto una prima visualizzazione degli attributi si nota che è utile passare gli attributi "disburse_time" e "planned_expiration_time" come date al momento dell'importazione.

In [11]:
loans = pd.read_csv(loans_path, parse_dates=["disburse_time", "planned_expiration_time"])

In [12]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [13]:
loans["duration"] = loans["planned_expiration_time"] - loans["disburse_time"]

Controllo per sicurezza quanti sono gli attributi nulli una volta fatta la differenza. In particolare è un numero abbastanza grosso, bisognerebbe quindi occuparsi di riempire i missing values.

In [14]:
loans.loc[loans['duration'].isnull() == True]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
64,251406,José Roan Group,Spanish,El grupo se llama “José Roan” y se ubica en l...,The group called “José Roan” is located in the...,825.0,825.0,funded,Poultry,Agriculture,Purchase young chickens and feed so that the a...,MX,Mexico,"COPAINALÁ, CHIAPAS",shared,0.2,MXN,130.0,2010-11-23 18:28:13.000 +0000,NaT,2010-11-18 08:00:00+00:00,2010-12-30 02:18:42.000 +0000,7.0,22,2,1,NaN,"female, female, female","true, true, true",monthly,field_partner,NaT
65,252537,Marylou,English,"Marylou S. is from the village of Kuguita, Mam...",NaN,175.0,175.0,funded,Personal Products Sales,Retail,to purchase additional inventory and as workin...,PH,Philippines,"Kuguita, Mambajao, Camiguin",not shared,NaN,PHP,128.0,2010-11-26 07:28:51.000 +0000,NaT,2010-11-25 08:00:00+00:00,2010-11-27 18:22:17.000 +0000,7.0,7,2,1,NaN,female,true,irregular,field_partner,NaT
66,253044,Flora,English,"Flora G. is from the village of Libona, Bukidn...",NaN,125.0,125.0,funded,Transportation,Transportation,To purchase tools needed to expand services pr...,PH,Philippines,"Libona, Bukidnon",not shared,NaN,PHP,128.0,2010-11-27 11:29:43.000 +0000,NaT,2010-11-25 08:00:00+00:00,2010-11-27 21:36:44.000 +0000,7.0,5,2,1,NaN,female,true,irregular,field_partner,NaT
67,254012,Maria Magdalena,Spanish,"María de 55 años, tiene una tienda de víveres ...","María, 55, has a grocery store in the front pa...",1000.0,1000.0,funded,General Store,Retail,"For rice, sugar, oil.",EC,Ecuador,Guayaquil (Fortin),not shared,NaN,USD,147.0,2010-11-30 22:38:46.000 +0000,NaT,2010-11-16 08:00:00+00:00,2010-12-24 21:33:46.000 +0000,12.0,36,1,1,NaN,female,true,monthly,field_partner,NaT
69,251336,Rober,Spanish,"Rober, de 32 años, está casado con Janeth, de ...","Rober, 32, is married to Janeth, 31, and they ...",375.0,375.0,funded,Knitting,Arts,To buy 15 bags of spools of thread.,PE,Peru,TARMA,shared,0.2,PEN,71.0,2010-11-23 15:22:39.000 +0000,NaT,2010-11-10 08:00:00+00:00,2010-11-24 07:18:30.000 +0000,12.0,15,3,2,NaN,male,true,monthly,field_partner,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419523,288,Grace Ngima,English,<i>This business is a member of Naretoi Women ...,NaN,1200.0,1200.0,funded,Cattle,Agriculture,She plans to maintain stock at a level of 10 s...,KE,Kenya,"Olturoto, Kajiado District",not shared,NaN,USD,6.0,2006-04-16 07:10:50.000 +0000,NaT,2005-04-14 05:27:55+00:00,2005-03-31 06:27:55.000 +0000,12.0,19,2,1,NaN,female,true,bullet,field_partner,NaT
1419524,1227124,Donna & Brighton,English,"In 1979, I graduated from the University of Wy...","In 1979, I graduated from the University of Wy...",800.0,10000.0,expired,Retail,Retail,"our community become stronger , healthier and...",US,United States,NaN,not shared,NaN,USD,NaN,2017-01-21 03:58:42.000 +0000,2017-02-09 23:07:54+00:00,NaT,NaN,36.0,9,1,1,NaN,female,true,bullet,direct,NaT
1419525,1089430,NaN,NaN,NaN,NaN,125.0,225.0,refunded,Grocery Store,Food,NaN,KE,Kenya,NaN,not shared,NaN,KES,NaN,2015-07-01 10:10:48.000 +0000,2015-09-21 08:06:19+00:00,NaT,NaN,6.0,3,1,1,NaN,NaN,NaN,bullet,direct,NaT
1419538,1235452,Tara,English,Currently homeless.\r\nComplete alchemist.\r\n...,Currently homeless.\r\nComplete alchemist.\r\n...,600.0,3000.0,expired,Technology,Services,me to set the foundation and get my business g...,US,United States,NaN,not shared,NaN,USD,NaN,2017-02-09 22:58:24.000 +0000,2017-02-25 20:21:21+00:00,NaT,NaN,24.0,3,1,1,NaN,female,true,bullet,direct,NaT


**3) Find the lenders that have funded at least twice.**

Per fare questa cosa ho gli attributi già pronti. Per prima cosa bisogna effettuare un merge tra loans_lenders_norm e loans_ per sapere se tutte le istanze presenti in loans_lenders norm corrispondano allo stato "funded", una volta fatto questo mi occupo di tenere solo le righe in cui lo stato è funded e di selezionare solo quelle con un conteggio maggiore di o uguale a 2. Per non sovraccaricare troppo la RAM utilizzo per il merge solo gli attributi che mi serviranno per le computazioni.

In [15]:
loans_lenders_merged = pd.merge(loans_lenders_norm, loans[["loan_id", "status"]], on="loan_id", how="left")

In [16]:
loans_lenders_merged_funded = loans_lenders_merged[loans_lenders_merged["status"] == "funded"]

In [19]:
num_loan = loans_lenders_merged_funded.groupby('lenders').count()['loan_id']
num_loan.loc[num_loan >= 2]

lenders
 000               39
 00000             38
 0002              70
 0101craign0101    71
 0132575            4
                   ..
zyrah8525           4
zyrorl              3
zzaman              9
zzanita             2
zzmcfate           54
Name: loan_id, Length: 993782, dtype: int64

**4) For each country, compute how many loans have involved that country as borrowers.**

Durante l'esecuzione di questo punto nasce la problematica causata dal gran numero di NaN presenti nell'attributo duration notato durante l'esecuzione del punto 2. Per effettuare un calcolo corretto bisogna quindi mettere delle condizioni, in particolare creo un nuovo dataframe che contenga le righe di loans con le seguenti caratteristiche:

- Non hanno valori null in "duration".
- Per quanto riguarda i valori in "planned_expiration_time" che siano minori di "disburse_time" li scambio  (**DA FARE**). 


In [21]:
loans_not_null = loans[loans["duration"].isnull() == False]
loans_not_null = loans_not_null[loans_not_null["planned_expiration_time"] > loans_not_null["disburse_time"]]

Ora mi occuppo di raggruppare per country e di contare solo quelli corretti.

In [24]:
num_loans_country = loans_not_null.groupby('country_name').count()['loan_id']

In [27]:
#Decommentare questa riga per la visualizzazione dei risultati.
#num_loans_country

**5) For each country, compute the overall amount of money borrowed.** 

Per farlo è gia a disposizione il dataset sistemato, bisogna solo raggruppare e sommare per paese.

In [29]:
loans_not_null.groupby('country_name')['loan_amount'].sum()

country_name
Afghanistan        6000.0
Albania         4307350.0
Armenia        20579275.0
Azerbaijan      6839400.0
Belize            65450.0
                  ...    
Vanuatu            9250.0
Vietnam        19907200.0
Yemen           3430100.0
Zambia          1658200.0
Zimbabwe        4838325.0
Name: loan_amount, Length: 84, dtype: float64

**6) Like the previous point, but expressed as a percentage of the overall amount lent.**

Allo stesso modo del punto precedente, basta stare attenti a fare il conto giusto.

In [33]:
overall_amount_lent = loans_not_null['loan_amount'].sum()
loans_country = pd.DataFrame(loans_not_null.groupby('country_name')['loan_amount'].sum())
loans_country['percentage'] = round(loans_country['loan_amount'] / overall_amount_lent * 100, 2)

In [37]:
#Decommentare questa riga per vedere i risultati.
#loans_country

Eseguo la somma per vedere che il risultato è unitario, ciò è vero a meno di approssimazioni che sono state svolte durante i conteggi:

In [35]:
round(loans_country['percentage'].sum(),2)

99.98

**7) Like the three previous points, but split for each year (with respect to disburse time).**

Il primo passaggio è quello di riprendere il dataset creato in precedenza e settare l'attributo "disburse_time" come indice in modo da poterci applicare il metodo Grouper di pandas.

In [38]:
loans_not_null["disburse_time"] =  pd.to_datetime(loans_not_null["disburse_time"])
loans_not_null = loans_not_null.set_index("disburse_time")
loans_by_year_sum = loans_not_null.groupby(["country_name", pd.Grouper(freq="Y")])["loan_amount"].sum().to_frame()

Ora faccio le stesse cose fatte sopra ma calcolando la percentuale.

In [40]:
loans_by_year_perc = loans_by_year_sum
loans_by_year_perc["loan_amount"] = loans_by_year_perc["loan_amount"]/overall_amount_lent*100
loans_by_year_perc = loans_by_year_perc.rename(columns={"loan_amount" : "loan_amount_perc"})

Faccio come sopra in modo da poter contare questa volta.

In [41]:
loans_by_year_count = loans_not_null.groupby(["country_name", pd.Grouper(freq="Y")])["loan_id"].count().to_frame()
loans_by_year_count = loans_by_year_count.rename(columns={"loan_id" : "count"})

Eseguo il merge tra i tre dataframe selezionando solo le colonne che non sono duplicate in modo da avere il minor spreco di memoria possibile.

In [45]:
country_statistics_by_year = pd.concat([loans_by_year_count, loans_by_year_sum, loans_by_year_perc], axis=1)
country_statistics_by_year = country_statistics_by_year.loc[:, ~country_statistics_by_year.columns.duplicated()]

In [46]:
#Decommentare questa riga per stampare i risultati.
#display(country_statistics_by_year.head())

**8) For each lender, compute the overall amount of money lent.**